In [1]:
'''
1. Take the loan data and process it as you did previously to build your linear regression model.
2. Break the data-set into 10 segments following the example provided here in KFold .
3. Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.
4. Comment on each of the performance metric you obtained.
'''

import pandas as pd
import numpy as np
import statsmodels.api as sm
loansData = pd.read_csv("https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv")
loansData.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length
81174,20000,20000,8.90%,36 months,debt_consolidation,14.90%,SC,MORTGAGE,6541.67,735-739,14,14272,2,< 1 year
99592,19200,19200,12.12%,36 months,debt_consolidation,28.36%,TX,MORTGAGE,4583.33,715-719,12,11140,1,2 years
80059,35000,35000,21.98%,60 months,debt_consolidation,23.81%,CA,MORTGAGE,11500.00,690-694,14,21977,1,2 years
15825,10000,9975,9.99%,36 months,debt_consolidation,14.30%,KS,MORTGAGE,3833.33,695-699,10,9346,0,5 years
33182,12000,12000,11.71%,36 months,credit_card,18.78%,NJ,RENT,3195.00,695-699,11,14469,0,9 years


In [2]:
loansData['Interest.Rate'] = loansData['Interest.Rate'].map(lambda x: float(x.replace("%","")))
loansData['Debt.To.Income.Ratio'] = loansData['Debt.To.Income.Ratio'].map(lambda x: float(x.replace("%",""))/100)
loansData['FICO.Range'] = loansData['FICO.Range'].map(lambda x: x.split("-"))
loansData['FICO.Score'] = loansData['FICO.Range'].map(lambda x: int(x[0]))
loansData['Loan.Length'] = loansData['Loan.Length'].map(lambda x: x.replace(" months","")) 

In [3]:
loansData.head()

,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,FICO.Score
81174,20000,20000,8.90,36,debt_consolidation,0.1490,SC,MORTGAGE,6541.67,"[735, 739]",14,14272,2,< 1 year,735
99592,19200,19200,12.12,36,debt_consolidation,0.2836,TX,MORTGAGE,4583.33,"[715, 719]",12,11140,1,2 years,715
80059,35000,35000,21.98,60,debt_consolidation,0.2381,CA,MORTGAGE,11500.00,"[690, 694]",14,21977,1,2 years,690
15825,10000,9975,9.99,36,debt_consolidation,0.1430,KS,MORTGAGE,3833.33,"[695, 699]",10,9346,0,5 years,695
33182,12000,12000,11.71,36,credit_card,0.1878,NJ,RENT,3195.00,"[695, 699]",11,14469,0,9 years,695


In [4]:
#print range(len(loansData.index))
loansData.reset_index(level=0, inplace=True)
loansData.rename(columns={'index':'Account'}, inplace=True)

In [5]:
loansData.head()

,Account,Amount.Requested,Amount.Funded.By.Investors,Interest.Rate,Loan.Length,Loan.Purpose,Debt.To.Income.Ratio,State,Home.Ownership,Monthly.Income,FICO.Range,Open.CREDIT.Lines,Revolving.CREDIT.Balance,Inquiries.in.the.Last.6.Months,Employment.Length,FICO.Score
0,81174,20000,20000,8.90,36,debt_consolidation,0.1490,SC,MORTGAGE,6541.67,"[735, 739]",14,14272,2,< 1 year,735
1,99592,19200,19200,12.12,36,debt_consolidation,0.2836,TX,MORTGAGE,4583.33,"[715, 719]",12,11140,1,2 years,715
2,80059,35000,35000,21.98,60,debt_consolidation,0.2381,CA,MORTGAGE,11500.00,"[690, 694]",14,21977,1,2 years,690
3,15825,10000,9975,9.99,36,debt_consolidation,0.1430,KS,MORTGAGE,3833.33,"[695, 699]",10,9346,0,5 years,695
4,33182,12000,12000,11.71,36,credit_card,0.1878,NJ,RENT,3195.00,"[695, 699]",11,14469,0,9 years,695


In [6]:
intrate = loansData['Interest.Rate']
loanamt = loansData['Amount.Requested']
fico = loansData['FICO.Score']

print intrate
# print loanamt
# print fico

0        8.90
1       12.12
2       21.98
3        9.99
4       11.71
5       15.31
6        7.90
7       17.14
8       14.33
9        6.91
10      19.72
11      14.27
12      21.67
13       8.90
14       7.62
15      15.65
16      12.12
17      10.37
18       9.76
19       9.99
20      21.98
21      19.05
22      17.99
23      11.99
24      16.82
25       7.90
26      14.42
27      15.31
28       8.59
29       7.90
        ...  
2470    22.95
2471     7.90
2472    22.45
2473    15.13
2474    18.75
2475    14.09
2476    14.09
2477     8.90
2478    11.71
2479    15.80
2480     6.03
2481     6.62
2482     7.51
2483    14.33
2484    10.16
2485    10.75
2486    17.27
2487    19.99
2488    15.81
2489    18.75
2490    11.71
2491     7.62
2492    10.08
2493    23.28
2494    14.65
2495    16.77
2496    14.09
2497    13.99
2498    12.42
2499    13.79
Name: Interest.Rate, dtype: float64


In [7]:
# first simple model
x = np.column_stack([loanamt,fico])
print x

[[20000   735]
 [19200   715]
 [35000   690]
 ..., 
 [10000   680]
 [ 6000   675]
 [ 9000   670]]


In [8]:
y = np.matrix(intrate)
# print y
# print intrate

In [9]:
X = sm.add_constant(x)
model = sm.OLS(intrate,X)
f = model.fit()

In [10]:
f.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Interest.Rate   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     2388.
Date:                Tue, 16 Feb 2016   Prob (F-statistic):               0.00
Time:                        21:48:03   Log-Likelihood:                -5785.4
No. Observations:                2500   AIC:                         1.158e+04
Df Residuals:                    2497   BIC:                         1.159e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         72.8828      0.988     73.734      0.000        70.945    74.821
x1             0.0002    6.3e-06     33.443      0.000         0.000     0.000
x2            -0.0884      0.001    -63.022      0.000        -0.091    -0.086
==============================================================================
Omnibus:                       69.496   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               77.811
Skew:                           0.379   Prob(JB):                     1.27e-17
Kurtosis:                       3.414   Cond. No.                     2.96e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.96e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
# second model includes monthly income as another variable
income = loansData['Monthly.Income']
print income.mean()
income = income.fillna(income.mean())
x1 = pd.concat([fico,income,loanamt], axis=1)
#print x1
x1 = np.column_stack([x1[x1.columns[0]],x1[x1.columns[1]],x1[x1.columns[2]]])
print x1

5688.93132053
[[   735.     6541.67  20000.  ]
 [   715.     4583.33  19200.  ]
 [   690.    11500.    35000.  ]
 ..., 
 [   680.     2166.67  10000.  ]
 [   675.     3500.     6000.  ]
 [   670.     3875.     9000.  ]]


In [12]:
X = sm.add_constant(x1)
model2 = sm.OLS(intrate, X)
f = model2.fit()
f.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Interest.Rate   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     1613.
Date:                Tue, 16 Feb 2016   Prob (F-statistic):               0.00
Time:                        21:48:03   Log-Likelihood:                -5774.2
No. Observations:                2500   AIC:                         1.156e+04
Df Residuals:                    2496   BIC:                         1.158e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         72.6330      0.986     73.690      0.000        70.700    74.566
x1            -0.0878      0.001    -62.526      0.000        -0.091    -0.085
x2         -6.357e-05   1.34e-05     -4.732      0.000     -8.99e-05 -3.72e-05
x3             0.0002   6.79e-06     32.831      0.000         0.000     0.000
==============================================================================
Omnibus:                       70.004   Durbin-Watson:                   1.975
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               78.142
Skew:                           0.383   Prob(JB):                     1.08e-17
Kurtosis:                       3.405   Cond. No.                     3.19e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.19e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
#the third model to include the debt/income ratio
ratio = loansData['Debt.To.Income.Ratio']
ratio = ratio.fillna(ratio.mean())
x2 = pd.concat([fico,income,loanamt,ratio], axis=1)
print x2
x2 = np.column_stack([x2[x2.columns[0]],x2[x2.columns[1]],x2[x2.columns[2]],x2[x2.columns[3]]])
print x2

      FICO.Score  Monthly.Income  Amount.Requested  Debt.To.Income.Ratio
0            735         6541.67             20000                0.1490
1            715         4583.33             19200                0.2836
2            690        11500.00             35000                0.2381
3            695         3833.33             10000                0.1430
4            695         3195.00             12000                0.1878
5            670         4891.67              6000                0.2005
6            720         2916.67             10000                0.2609
7            705        13863.42             33500                0.1470
8            685         3150.00             14675                0.2692
9            715         5000.00              7000                0.0710
10           670         3575.00              2000                0.1029
11           665         4250.00             10625                0.1254
12           670        14166.67             28000 

In [21]:
X = sm.add_constant(x2)
model3 = sm.OLS(intrate,X)
f = model3.fit()
f.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Interest.Rate   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.660
Method:                 Least Squares   F-statistic:                     1215.
Date:                Tue, 16 Feb 2016   Prob (F-statistic):               0.00
Time:                        21:59:43   Log-Likelihood:                -5770.2
No. Observations:                2500   AIC:                         1.155e+04
Df Residuals:                    2495   BIC:                         1.158e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         73.5433      1.035     71.061      0.000        71.514    75.573
x1            -0.0887      0.001    -61.798      0.000        -0.091    -0.086
x2          -7.12e-05   1.37e-05     -5.205      0.000      -9.8e-05 -4.44e-05
x3             0.0002   6.89e-06     32.879      0.000         0.000     0.000
x4            -1.9407      0.682     -2.846      0.004        -3.278    -0.603
==============================================================================
Omnibus:                       70.527   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               79.018
Skew:                           0.383   Prob(JB):                     6.94e-18
Kurtosis:                       3.415   Cond. No.                     3.46e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.46e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [24]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
f = regr.fit(X, intrate)
print f.coef_
print f.intercept_

[  0.00000000e+00  -8.86573992e-02  -7.12006163e-05   2.26447864e-04
  -1.94073594e+00]
73.5432570721


In [32]:
#=============2. Break the data-set into 10 segments following the example provided here in KFold .===========
from sklearn.cross_validation import KFold
kf = KFold(2499, 10)

actual = []
pred = []
for train, test in kf:
    #print("Train: %s, Test: %s" % (train, test))
    X_train, X_test = X[train], X[test]
    y_train, y_test = intrate[train], intrate[test]
    #print X_train, y_train
    f = regr.fit(X_train, y_train)
    predict = regr.predict(X_test)
    actual.append(y_train)
    pred.append(predict)
    
print actual
#print pred

[250      8.94
251      7.49
252     11.03
253     10.16
254     16.89
255     14.35
256     14.09
257     15.57
258     14.59
259     16.29
260      5.99
261     14.65
262      7.90
263     10.99
264      9.62
265     20.50
266      7.62
267      6.91
268     17.77
269     15.31
270     10.16
271     10.36
272     17.27
273     15.27
274     17.49
275     17.77
276     11.54
277     12.69
278     10.74
279     13.99
        ...  
2469    10.74
2470    22.95
2471     7.90
2472    22.45
2473    15.13
2474    18.75
2475    14.09
2476    14.09
2477     8.90
2478    11.71
2479    15.80
2480     6.03
2481     6.62
2482     7.51
2483    14.33
2484    10.16
2485    10.75
2486    17.27
2487    19.99
2488    15.81
2489    18.75
2490    11.71
2491     7.62
2492    10.08
2493    23.28
2494    14.65
2495    16.77
2496    14.09
2497    13.99
2498    12.42
Name: Interest.Rate, dtype: float64, 0        8.90
1       12.12
2       21.98
3        9.99
4       11.71
5       15.31
6        7.90
7       17

In [16]:
#==3. Compute each of the performance metric (MAE, MSE or R2) for all the folds. The average would be the performance of your model.==
